<a href="https://colab.research.google.com/github/jonnells/cmpe255-project-grp/blob/main/Yelp_On_The_Go.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import os
import json

In [2]:
yelp_dataset_fp = 'yelp_dataset/'

In [3]:
user_data = []
with open(yelp_dataset_fp + 'yelp_academic_dataset_user.json', encoding="utf8") as fin:
    print('Reading yelp_academic_dataset_user.json')
    for line in fin:
        user_data.append(json.loads(line))

Reading yelp_academic_dataset_user.json


In [4]:
review_data = []
with open(yelp_dataset_fp + 'yelp_academic_dataset_review.json', encoding="utf8") as fin:
    print('Reading yelp_academic_dataset_review.json')
    for line in fin:
        review_data.append(json.loads(line))

Reading yelp_academic_dataset_review.json


In [6]:
print(str(len(user_data)) + ' user records loaded')
print(str(len(review_data)) + ' review records loaded')

2189457 user records loaded
8635403 review records loaded
